In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import re

import geopandas as gpd
import requests,json


from shapely import geos
from shapely.geometry import Point
import fiona
from fiona.crs import from_epsg,from_string

from geopandas import GeoDataFrame
from shapely.geometry import Polygon
from geopandas import GeoSeries


from shapely import geometry
from shapely.geometry import shape,LineString,Point,Polygon

import osmnx as ox
import networkx as nx

import folium

from folium import IFrame

In [3]:
def rest(address):
    try:
        url = 'https://api.tomtom.com/search/2/geocode/{}.json?&key=h2n93GznJsnsDHtv2vPpdT9GmJijp9o9&countrySet=TWN&language=zh-TW&limit=1'.format(address)
        response = requests.get(url)
        data = response.text
        js = json.loads(str(data))
        return js['results'][0]['position']
    except Exception as e:
        print(f'An Error occurred: {e}')
        return {'lat':0,'lon':0}
    
def trans(gdf):
    gdf.crs = {'init' :'epsg:3824'} # 避免資料沒設，這邊再重新給一次
    gdf=gdf.to_crs({'init' :'epsg:4326'})
    return gdf

def utf2asc(s):
    return str(str(s).encode('ascii', 'xmlcharrefreplace'))[2:-1]

heading3 = """<h3>{}</h3>""".format

In [4]:
import warnings
warnings.filterwarnings("ignore")

# 設定row data的資料夾

In [5]:
import os
os.chdir(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data')

In [8]:
filename='HD_ALL'
save_file='HD'
save_year='ALL'

# SQL有正確地址的資料

In [9]:
df_01=pd.read_csv(filename+"一般卡會員(有完整地址).csv",error_bad_lines=False,header=None)
df_01.columns=['id','addr','lon','lat']
df_01.shape

(8646, 4)

## 第一次抓取 村里

In [10]:
pattern = re.compile(r'(..[市,縣])(.{1,2}[市,區,鄉,鎮])(.{2,3}[里,村])')
location=df_01.addr.apply(lambda x:pattern.findall(x))

addr01=[]
addr02=[]
addr03=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0][0])
        addr02.append(i[0][1])
        addr03.append(i[0][2])        
    else:
        addr01.append(np.nan)
        addr02.append(np.nan)
        addr03.append(np.nan)  

In [11]:
df_01['縣市']=addr01
df_01['行政區']=addr02
df_01['村里']=addr03

### 第一次 有村里 資料

In [12]:
df_02=df_01.dropna()
df_02.shape

(8644, 7)

### 村里邊界資料

In [13]:
gdf=gpd.read_file(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\geo data\VILLAGE_MOI_1090423.shp',encoding='utf-8')
gdf=gdf.loc[:,['COUNTYNAME','TOWNNAME','VILLNAME','geometry']]
gdf=trans(gdf)
gdf=gdf.dropna().reset_index(drop=True)
print(gdf.shape)

gdf.columns=['縣市','行政區','村里','geometry']
gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))

df_02['縣市']=list(df_02['縣市'].apply(lambda x:x.replace('臺','台')))
df_02['行政區']=list(df_02['行政區'].apply(lambda x:x.replace('臺','台')))
df_02['村里']=list(df_02['村里'].apply(lambda x:x.replace('臺','台')))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舘','館'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('双','雙'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))
gdf.head()

(7760, 4)


,縣市,行政區,村里,geometry
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498..."
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492..."
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577..."
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557..."
4,連江縣,南竿鄉,津沙村,"POLYGON ((119.91684 26.14109, 119.91683 26.141..."


### 檢查1 & 字元調整

In [14]:
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖部里','糖廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖?里','糖廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚?里','磚磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚瑤里','磚磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下?里','下廍里'))

df_02.loc[df_02['行政區']=='彰化市',['村里']]=df_02.loc[df_02['行政區']=='彰化市',['村里']].apply(lambda x:x.replace('南?里','南瑶里'))
df_02.loc[df_02['行政區']=='官田區',['村里']]=df_02.loc[df_02['行政區']=='官田區',['村里']].apply(lambda x:x.replace('南?里','南廍里'))
#df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?里','南瑶里'))


df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙?里','崙峯里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?子里','廍子里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('石?村','石硦村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('上?里','上館里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('永?里','永館里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?村','瓦磘村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?里','瓦磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('五?里','五峯里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?行里','塩行里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?興里','塩興里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('公?里','公館里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?洲里','塩洲里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?龍里','雙龍里'))

#df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?林里','雞林里'))
df_02.loc[df_02['行政區']=='竹東鎮',['村里']]=df_02.loc[df_02['行政區']=='竹東鎮',['村里']].apply(lambda x:x.replace('?林里','雞林里'))

df_02.loc[df_02['行政區']=='大園區',['村里']]=df_02.loc[df_02['行政區']=='大園區',['村里']].apply(lambda x:x.replace('?林里','菓林里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?寮里','獇寮里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('新?里','新廍里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('蔗?里','蔗廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?埔里','坔埔里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?榔里','槺榔里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?江里','晋江里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?福村','雙福村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('爪?里','爪峯里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舊?村','舊館村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?村','南館村'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('灰?里','灰磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('三?村','三廍村'))


df_02.loc[df_02['行政區']=='將軍區',['村里']]=df_02.loc[df_02['行政區']=='將軍區',['村里']].apply(lambda x:x.replace('忠興里','忠嘉里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下部里','下廍里'))
df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']]=df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']].apply(lambda x:x.replace('?子村','廍子村'))
df_02.loc[df_02['行政區']=='四湖鄉',['村里']]=df_02.loc[df_02['行政區']=='四湖鄉',['村里']].apply(lambda x:x.replace('?子村','萡子村'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?田里','塩田里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('山?里','山脚里'))

df_02.loc[df_02['行政區']=='北埔鄉',['村里']]=df_02.loc[df_02['行政區']=='北埔鄉',['村里']].apply(lambda x:x.replace('水?村','水祭村'))
df_02.loc[df_02['行政區']=='北埔鄉',['村里']]=df_02.loc[df_02['行政區']=='北埔鄉',['村里']].apply(lambda x:x.replace('水？村','水祭村'))
df_02.loc[df_02['行政區']=='瑞芳區',['村里']]=df_02.loc[df_02['行政區']=='瑞芳區',['村里']].apply(lambda x:x.replace('?洞里','濂洞里'))

df_02.loc[df_02['行政區']=='埔心鄉',['村里']]=df_02.loc[df_02['行政區']=='埔心鄉',['村里']].apply(lambda x:x.replace('埤?村','埤脚村'))
df_02.loc[df_02['行政區']=='彰化市',['村里']]=df_02.loc[df_02['行政區']=='彰化市',['村里']].apply(lambda x:x.replace('寶?里','寶廍里'))

In [15]:
site1=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].groupby(['村里'])['行政區'].nunique().reset_index(name='n').sort_values(by='n',ascending=False).reset_index(drop=True)
site1.head(10)

,村里,n
0,?北里,1
1,上里,1


In [16]:
number=0
for i in site1[site1.n>0]['村里']:
    print(number,i, df_02[df_02['村里']==i]['行政區'].unique())
    number+=1

0 ?北里 ['左營區']
1 上里 ['竹東鎮']


In [17]:
'''
site2=site1[site1.n>0]['村里'][48]
site3=df_02[df_02['村里']==site2]['行政區'].unique()

for i in site3:
    print(i,list(gdf[gdf['行政區']==i]['村里'][gdf[gdf['行政區']==i]['村里'].str.contains('龜')]))
'''

"\nsite2=site1[site1.n>0]['村里'][48]\nsite3=df_02[df_02['村里']==site2]['行政區'].unique()\n\nfor i in site3:\n    print(i,list(gdf[gdf['行政區']==i]['村里'][gdf[gdf['行政區']==i]['村里'].str.contains('龜')]))\n"

In [18]:
#df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])]
#df_02[(df_02['行政區']+df_02['村里'])=='埔心鄉埤脚村']

In [19]:
#gdf[(gdf['行政區']+gdf['村里'])=='埔心鄉埤霞村']

In [20]:
#df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].groupby(['行政區','村里']).count().sort_values(by='id',ascending=False).head(15)

In [21]:
#gdf[gdf['村里']=='三廍村']

In [22]:
#df_02[df_02['村里']=='五?里']['行政區'].unique()

### 第一次抓取 村里 與 geo data 合不合

In [23]:
#合
df_02_1=df_02[(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)

#不合
df_02_2=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_02_1.shape)
print(df_02_2.shape)

(8642, 7)
(2, 7)


### 不合的資料用經緯度 與村里邊界資料mapping

In [24]:
if df_02_2.shape[0]>0:
    df_02_2.lon=df_02_2.lon.apply(lambda x:float(x))
    df_02_2.lat=df_02_2.lat.apply(lambda x:float(x))
    geom=[Point(data.lon,data.lat) for idx, data in df_02_2.iterrows()]
    crs = {'init': 'epsg:4326'}
    df_02_2 = gpd.GeoDataFrame(df_02_2, crs=crs, geometry=geom)
    print(df_02_2.shape) 
    df_02_3 = gpd.tools.sjoin(df_02_2,gdf, how="inner", op='intersects')
    print(df_02_3.shape)
    df_02_3=df_02_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
    df_02_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
    print(df_02_3.head(9))
    
    df_02=pd.concat([df_02_1,df_02_3]).reset_index(drop=True)
    print('合併成功',df_02.shape)
else:
    print('不須合併',df_02_2.shape)
    df_02=df_02_1.reset_index(drop=True)
    print(df_02.shape)

(2, 8)
(2, 12)
               id                                               addr  \
0  20180116000456  高雄市左營區?北里5鄰海平路60號                             ...   
1  19991027004557  新竹縣竹東鎮上里18鄰和江街330巷6號                         ...   

          lon        lat   縣市  行政區   村里  
0  120.288192  22.686029  高雄市  左營區  廍北里  
1  121.089348  24.723878  新竹縣  竹東鎮  上館里  
合併成功 (8644, 7)


## 合併 合與不合的資料
## 獲得完整的第一次抓取 村里

## 第一次抓取 無村里的資料

In [25]:
df_03=df_01[-df_01.id.isin(df_02.id)].dropna(subset=['lon'])
df_03.shape

(2, 7)

### 第二次抓取 村里

In [26]:
pattern = re.compile(r'(..[市,縣])(.{2,3}[市,區,鄉,鎮])(.{1,2}[里,村])')
location=df_03.addr.apply(lambda x:pattern.findall(x))

addr01=[]
addr02=[]
addr03=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0][0])
        addr02.append(i[0][1])
        addr03.append(i[0][2])        
    else:
        addr01.append(np.nan)
        addr02.append(np.nan)
        addr03.append(np.nan)  

In [27]:
df_03['縣市']=addr01
df_03['行政區']=addr02
df_03['村里']=addr03

### 第二次抓取 村里 與 geo data 合不合

In [28]:
#合
df_03_1=df_03[(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_03_1.shape)

#不合
df_03_2=df_03[-(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_03_2.shape)


(0, 7)
(2, 7)


### 不合的資料用經緯度 與村里邊界資料mapping

In [29]:
def to_float(x):
    try:
        return float(x)
    except:
        return 0

In [30]:
df_03_2.lon=df_03_2.lon.apply(to_float)
df_03_2.lat=df_03_2.lat.apply(to_float)

In [31]:
if df_03_2.shape[0]>0:
    geom=[Point(data.lon,data.lat) for idx, data in df_03_2.iterrows()]
    crs = {'init': 'epsg:4326'}
    df_03_2 = gpd.GeoDataFrame(df_03_2, crs=crs, geometry=geom)
    print(df_03_2.shape)
    df_03_3 = gpd.tools.sjoin(df_03_2,gdf, how="inner", op='intersects')
    print(df_03_3.shape)
    df_03_3=df_03_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
    df_03_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
    print(df_03_3.shape)
else:
    df_03_3=df_03_2.copy()

(2, 8)
(2, 12)
(2, 7)


In [32]:
df_03_3.head(5)

,id,addr,lon,lat,縣市,行政區,村里
0,20040409000169,臺南市新化區(那)拔里9鄰牧場1之20號 ...,120.331038,23.061009,台南市,新化區,[那]拔里
1,19990829000715,台北市內湖區 ...,121.588470,25.069090,台北市,內湖區,湖興里


In [33]:
df_03_4=df_03_2[-df_03_2.id.isin(df_03_3.id)].loc[:,['id','addr','lon','lat','縣市','行政區','村里']]
print('df_03_4',df_03_4.shape)
print('df_03',df_03.shape)
df_03=pd.concat([df_03_1,df_03_3]).reset_index(drop=True)
print('df_03_1',df_03_1.shape)
print('df_03_3',df_03_3.shape)
print('df_03',df_03.shape)
df_03=pd.concat([df_03,df_03_4]).reset_index(drop=True)
print('df_03',df_03.shape)

df_03_4 (0, 7)
df_03 (2, 7)
df_03_1 (0, 7)
df_03_3 (2, 7)
df_03 (2, 7)
df_03 (2, 7)


## 合併 合與不合的資料
## 獲得完整的第二次抓取 村里

## 合併 第一次抓取 有村里 與 無村里 資料，獲得SLQ有正確地址的資料

In [34]:
df_01=pd.concat([df_02,df_03]).reset_index(drop=True)
df_01.shape

(8646, 7)

# 存檔

In [35]:
df_01.to_csv('C:/Users/RT009223/TestriteGroup/滲透率專案/02_adjusted data/'+save_file+'會員位置(經整理_配對村里)_一般卡_'+save_year+'.txt',index=0)

# SLQ無正確地址的資料

In [36]:
df_04=pd.read_excel(filename+'一般卡會員(無完整地址).xlsx')
df_04.shape

(1928, 3)

In [37]:
df_04['addr']=df_04.addr.apply(lambda x:str(x))

# 區分有可用的地址 與 無可用的地址
## 有可用的地址

In [38]:
df_04_1=df_04[df_04.addr.apply(lambda x:len(x)>9)].reset_index(drop=True)
df_04_1.shape

(802, 3)

In [39]:
pattern = re.compile(r'..[市,縣].+[路,街].+')
location=df_04_1.addr.apply(lambda x:pattern.findall((x)))

addr01=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0])      
    else:
        addr01.append(np.nan)

In [40]:
df_04_1.addr=addr01
df_04_1=df_04_1.dropna(subset=['addr']).reset_index(drop=True)
df_04_1.shape

(666, 3)

In [41]:
df_04_1['lon']=float(0)
df_04_1['lat']=float(0)
df_04_1['check']=0
df_04_1.head()

,id,addr,zone,lon,lat,check
0,20200116000105,台北市士林區基河路22號5樓,111.0,0.0,0.0,0
1,20100619000186,新竹市新竹市香山區海山里大湖路71巷3弄78號,300.0,0.0,0.0,0
2,20151018000209,桃園市桃園區永安街403巷32號9樓,330.0,0.0,0.0,0
3,20020412000065,台中市南屯區工業區３８路１００巷３號５Ｆ,408.0,0.0,0.0,0
4,20000101004187,台中市北屯區崇德11路41之1號1樓,406.0,0.0,0.0,0


## 與現有 搜尋過 經緯度的資料 比對

In [42]:
df_get_back=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\get back.xlsx')
print(df_get_back.shape)
df_get_back.head()

(133014, 6)


,id,addr,zone,lon,lat,check
0,30012476701000,新北市貢寮區鳥崗街13號,228.0,122.00193,25.01326,1
1,20051030000358,新北市貢寮區福興村東興街４７號,228.0,121.99057,25.01205,1
2,20011022000749,新北市貢寮區春福連村臉香蘭街９號,228.0,121.98674,25.01985,1
3,20051219001868,新北市新店區東興街22號3樓DM退件,231.0,121.94981,25.01872,1
4,19980808000468,新北市貢寮區惠牴街25-2號,228.0,121.94576,25.01659,1


In [43]:
df_get_back=df_get_back=df_get_back.loc[:,['id','lon','lat','check']]

In [44]:
df_04_1=pd.merge(df_04_1.loc[:,['id','addr','zone']],df_get_back,on='id',how='left')

In [45]:
df_04_1.lon[df_04_1.lon.isna()]=0
df_04_1.lat[df_04_1.lat.isna()]=0
df_04_1.check[df_04_1.check.isna()]=0

In [46]:
df_04_1.head()

,id,addr,zone,lon,lat,check
0,20200116000105,台北市士林區基河路22號5樓,111.0,0.00000,0.00000,0.0
1,20100619000186,新竹市新竹市香山區海山里大湖路71巷3弄78號,300.0,120.91935,24.76185,1.0
2,20151018000209,桃園市桃園區永安街403巷32號9樓,330.0,0.00000,0.00000,0.0
3,20020412000065,台中市南屯區工業區３８路１００巷３號５Ｆ,408.0,120.58976,24.17659,1.0
4,20000101004187,台中市北屯區崇德11路41之1號1樓,406.0,0.00000,0.00000,0.0


## 與郵遞區號配對

In [47]:
df_zone=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\郵遞區代碼.xlsx')

In [48]:
df_04_1=pd.merge(df_04_1,df_zone,on='zone',how='left')

In [49]:
geom=[Point(data.lon,data.lat) for idx, data in df_04_1.iterrows()]
crs = {'init': 'epsg:4326'}
df_04_1 = gpd.GeoDataFrame(df_04_1, crs=crs, geometry=geom)
df_04_1.shape

(666, 9)

In [50]:
df_04_1= gpd.tools.sjoin(df_04_1,gdf, how="left", op='intersects')
df_04_1.shape

(666, 13)

In [51]:
df_04_1.head()

,id,addr,zone,lon,lat,check,縣市_left,行政區_left,geometry,index_right,縣市_right,行政區_right,村里
0,20200116000105,台北市士林區基河路22號5樓,111.0,0.00000,0.00000,0.0,台北市,士林區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
1,20100619000186,新竹市新竹市香山區海山里大湖路71巷3弄78號,300.0,120.91935,24.76185,1.0,NaN,NaN,POINT (120.91935 24.76185),2388.0,新竹市,香山區,海山里
2,20151018000209,桃園市桃園區永安街403巷32號9樓,330.0,0.00000,0.00000,0.0,桃園市,桃園區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
3,20020412000065,台中市南屯區工業區３８路１００巷３號５Ｆ,408.0,120.58976,24.17659,1.0,台中市,南屯區,POINT (120.58976 24.17659),2149.0,台中市,西屯區,協和里
4,20000101004187,台中市北屯區崇德11路41之1號1樓,406.0,0.00000,0.00000,0.0,台中市,北屯區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN


In [52]:
# 無 map 無 zone
df_04_1_1=df_04_1[(df_04_1['行政區_left'].isna()&df_04_1['村里'].isna())]
print(df_04_1_1.shape)

# 無 map 有 zone
df_04_1_2=df_04_1[(-(df_04_1['行政區_left'].isna())&(df_04_1['村里'].isna()))]
print(df_04_1_2.shape)

# 有 map 無有 zone
df_04_1_3=df_04_1[(df_04_1['行政區_left'].isna()&(-df_04_1['村里'].isna()))]
print(df_04_1_3.shape)

# 有 map 有 zone
df_04_1_4=df_04_1[(-(df_04_1['行政區_left'].isna())&(-df_04_1['村里'].isna()))]
print(df_04_1_4.shape)

(16, 13)
(423, 13)
(74, 13)
(153, 13)


###  有 map 有 zone 合不合

In [53]:
df_04_1_4_1=df_04_1_4[(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
print(df_04_1_4_1.shape)

df_04_1_4_2=df_04_1_4[-(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
print(df_04_1_4_2.shape)

(125, 13)
(28, 13)


### 調整各組資料

In [54]:
df_04_1_1=df_04_1_1.loc[:,['id']]

df_04_1_2=df_04_1_2.loc[:,['id','縣市_left','行政區_left']]
df_04_1_2.columns=['id','縣市','行政區']

df_04_1_3=df_04_1_3.loc[:,['id','縣市_right','行政區_right','村里']]
df_04_1_3.columns=['id','縣市','行政區','村里']

df_04_1_4_1=df_04_1_4_1.loc[:,['id','縣市_right','行政區_right','村里']]
df_04_1_4_1.columns=['id','縣市','行政區','村里']

df_04_1_4_2=df_04_1_4_2.loc[:,['id','縣市_left','行政區_left']]
df_04_1_4_2.columns=['id','縣市','行政區']

In [55]:
df_04_2=pd.concat([df_04_1_4_1,df_04_1_3,df_04_1_4_2,df_04_1_2,df_04_1_1])
df_04_2.shape

(666, 4)

In [56]:
df_04_2.drop_duplicates(subset=['id']).shape

(666, 4)

## 無可用地址

In [57]:
df_04_3=df_04[-df_04.id.isin(df_04_2.id)]
df_04_3.shape

(1262, 3)

In [58]:
df_04_3=pd.merge(df_04_3,df_zone,on='zone',how='left')

In [59]:
df_04_3_1=df_04_3[-df_04_3['行政區'].isna()]
print(df_04_3_1.shape)

df_04_3_2=df_04_3[df_04_3['行政區'].isna()]
print(df_04_3_2.shape)

(167, 5)
(1095, 5)


In [60]:
df_04_3_1=df_04_3_1.loc[:,['id','縣市','行政區']]
df_04_3_2=df_04_3_2.loc[:,['id']]

# 完整 SLQ無正確地址的資料

In [61]:
df_04=pd.concat([df_04_2,df_04_3_1,df_04_3_2])
print(df_04.shape)
print(df_04.drop_duplicates(subset=['id']).shape)

(1928, 4)
(1928, 4)


# 存檔

In [62]:
df_04.to_csv('C:/Users/RT009223/TestriteGroup/滲透率專案/02_adjusted data/'+save_file+'會員位置(無正確地址的資料)_一般卡_'+save_year+'.txt',index=0)